**Run 3 analysis**

**Framework version January 2024**
- Log :
    - understood bug in data (the problem was in CRAB not here)
    - added Snapshot 
- Planned update :
    - standalone code, prepare a couple of input parameters and make the code working with a single command
    
    
__________________________________________

**Code**

Folder definition on Tier:
- in the main folder */acagnott/* added folder 'remote_folder_name';
- in "Snapshots" added the folder 'remote_subfolder_name';
- in the subfolder through dask the snapshot will be copied with name "snap_"+label+"_*.root"


Es: se lancio "DataMETA_2018", gli snapshot vengono salvati in ../acagnott/Snapshot/20231229/snap_DataMET_2018_*.root
se viene lanciato "QCD_2018" viene creata la cartella /acagnott/Snapshot/20231229/snap_QCDHT_100to200_2018_*.root e così via per ogni components

---> Viene usato solo il giorno in modo che tutti i sample lanciati lo stesso giorno verranno salvati nella stessa cartella con nomi diversi, visto che vengono lanciati in momenti diversi della giornata lo stesso tipo di job. Forse va modificato il formato se i singoli job iniziano a durare più di un giorno dato che viene comunque lanciato un sampel per volta (in tal caso verrebbero salvati in cartelle diverse. Si potrebbe pensare di mettere la data a mano, cioé invece di usare datetime.now() si potrebbe inserire la data manualmente per fare in modo di mettere tutti i file nella stessa folder. 

In [1]:
from datetime import datetime
sched_port = 20985 #Dask port
nmaxpartition = 28
distributed = True#False#
do_histos = True
hist_folder = "run2022_trigSF"
do_snapshot = False
remote_subfolder_name = datetime.now().strftime("%Y%m%d") #20231229

in_dataset = [
    # "DataMETA_2018", 
    # "DataMETB_2018", 
    # "DataMETC_2018",
    # "DataMETD_2018",
    # "DataSingleMuA_2018", 
    # "DataSingleMuB_2018", 
    # "DataSingleMuC_2018",
    # "DataSingleMuD_2018",
    
    # "TprimeToTZ_700_2018",
    # "TprimeToTZ_1000_2018", 
    # "TprimeToTZ_1800_2018",
    
    # "QCD_2018",
    # "TT_2018",
    # "ZJetsToNuNu_2018",
    # "WJets_2018",
    
    # "QCDHT_100to200_2018", "QCDHT_200to300_2018","QCDHT_300to500_2018", "QCDHT_500to700_2018", "QCDHT_700to1000_2018","QCDHT_1000to1500_2018", "QCDHT_1500to2000_2018","QCDHT_2000toInf_2018","TT_hadr_2018", "TT_semilep_2018", "TT_Mtt700to1000_2018", "TT_Mtt1000toInf_2018","ZJetsToNuNu_HT100to200_2018", "ZJetsToNuNu_HT200to400_2018", "ZJetsToNuNu_HT400to600_2018", "ZJetsToNuNu_HT600to800_2018", "ZJetsToNuNu_HT800to1200_2018", "ZJetsToNuNu_HT1200to2500_2018", "ZJetsToNuNu_HT2500toInf_2018""WJetsHT100to200_2018", "WJetsHT200to400_2018", "WJetsHT400to600_2018", "WJetsHT600to800_2018", "WJetsHT800to1200_2018", "WJetsHT1200to2500_2018", "WJetsHT2500toInf_2018"
    
    # "WJets_2022",
    # "ZJetsToNuNu_2022",
    
    "QCD_2022",
    # "ZJetsToNuNu_2jets_2022",
    # "TT_2022",
    # "WJets_2jets_2022"
    # "DataEGamma_2022"

]

# "event", "run", 
branches = {"HT_eventHT", "Top_mass", "Top_pt", "Top_score", "Top_isolationPtJetsdR04", "Top_isolationPtJetsdR06", "Top_isolationPtJetsdR08", "Top_isolationPtJetsdR12", "Top_isolationNJetsdR04", "Top_isolationNJetsdR06", "Top_isolationNJetsdR08", "Top_isolationNJetsdR12",
            "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60", "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight",  
            "MET_pt", "nVetoMuon", "nVetoElectron", "nJetBtag", 
            "nGoodJet", "nTightElectron", "nTightMuon", "MT"}

In [2]:
import ROOT
import os
from utils.samples import *
from utils.variables import *
import json
import numpy as np
import math
from dask.distributed import Client
ROOT.RDF.Experimental.Distributed.open_files_locally = False

os.environ['X509_CERT_DIR'] = "/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
os.environ['X509_USER_PROXY'] = "/tmp/x509up_u0"
print(os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR"))


if distributed:
    nfiles_max = 1000
else:
    nfiles_max = 1  #######

# Cosa aggiungere, modificare la cartella sul tier in questo modo ../acagnott/Snapshot_rdf/*dataset_name*/*data di processamente con orario*/
# insomma come fa crab


if do_histos: print("You are producing histograms")
if do_snapshot: print("You are producing snapshot")

remote_folder_name = "Snapshots"
#output histos folder
folder = "./results/"+hist_folder+"/"
# eos_folder = "/eos/home-a/acagnott/DarkMatter/nosynch/"+hist_folder

if do_snapshot and remote_subfolder_name == datetime.now().strftime("%Y%m%d") and distributed: 
    print("You are naming the tier subfolder using the current day \n")
    print("Snapshots folder name : ~/acagnott/{}/{}".format(remote_folder_name, remote_subfolder_name))
elif do_snapshot and distributed:
    print("You are naming the tier subfolder manually")
    print("Snapshots folder name : ~/acagnott/{}/{}".format(remote_folder_name, remote_subfolder_name))
elif do_snapshot:
    print("You are saving snapshots in local")
    print("folder name : " + folder)


if do_histos : 
    print("local folder histos: {}".format(folder))
    # print("At the end of the job the histos will be transfer to eos storage")
    # print("eos folder : {}".format(eos_folder))
if not os.path.exists(folder):
    os.mkdir(folder)
repohisto = folder+"plots/"
if not os.path.exists(repohisto):
    os.mkdir(repohisto)

Welcome to JupyROOT 6.27/01
/tmp/x509up_u0 /cvmfs/grid.cern.ch/etc/grid-security/certificates/
You are producing histograms
local folder histos: ./results/run2022_trigSF/


In [3]:
# Creating folders on Tier
if do_snapshot and distributed:
    tier_main_folder = "davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/"
    os.popen("davix-mkdir davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{} -E /tmp/x509up_u0 --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/".format(remote_folder_name))
    os.popen("davix-mkdir davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{}/{} -E /tmp/x509up_u0 --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/".format(remote_folder_name, remote_subfolder_name))
    
# transfer function for dask worker
def transfer_to_tier(dask_worker):
    import os
    os.popen('for filename in snap_*.root; do davix-put $filename davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{}/{}/$filename -E ./proxy --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/; done'.format(remote_folder_name, remote_subfolder_name))
    return True #, os.popen("for filename in ./test_*.root; do echo $filename; done").read()

- Import of utils from variables.py
Cut (if any), Regions, Variables

- syncro between in_dataset and sample_dict (from sample.py) to syncronize labels and ather featurs of the dataset (as sigma if needed)
- import of samples_dict.json to load files list (path to reach them on tier)


In [4]:
cut = requirements # ---> see variables.py

hlt_met = "(HLT_PFMET120_PFMHT120_IDTight || HLT_PFMETNoMu120_PFMHTNoMu120_IDTight)"
# hlt_mu  = "(HLT_IsoMu24 || HLT_Mu50)"
hlt_ele = "(HLT_Ele32_WPTight_Gsf || HLT_Ele115_CaloIdVT_GsfTrkIdT || HLT_Photon200)"

regions_def = {
    "orthogonalPreselR_Ntot"     :  hlt_ele +" && PuppiMET_T1_pt_nominal>100 && nTightElectron>0",
    "orthogonalPreselR_Npass"    :  hlt_ele+" && "+ hlt_met +" && PuppiMET_T1_pt_nominal>100 && nTightElectron>0",
    "orthogonalPreselR2_Ntot"     :  hlt_ele +" && PuppiMET_T1_pt_nominal>100 && nTightElectron>0",
    "orthogonalPreselR2_Npass"    :  hlt_ele+" && "+ hlt_met +" && PuppiMET_T1_pt_nominal>100 && MinDelta_phi>0.6 && nTightElectron>0",
    "orthogonalPreselR_CR"       :  hlt_ele+" && "+ hlt_met +" && PuppiMET_T1_pt_nominal>100 && MinDelta_phi>0.6 && nTightElectron==0",
}
print("Regions to book: ")
for r in regions_def.keys():
    print("  "+r)
    
sample_file = open("utils/dict_samples_2022.json", "rb")
samples = json.load(sample_file)
sample_file.close()

var = []
var.append(variable(name = "PuppiMET_pt", title= "p_{T}^{miss}(Puppi) [GeV]", nbins = 8, xmin = 100, xmax=800))
var.append(variable(name = "PuppiMET_phi", title= "MET #phi (Puppi) [GeV]", nbins = 6, xmin = -math.pi, xmax=math.pi))
var.append(variable(name = "PuppiMET_T1_pt_nominal", title= "p_{T}^{miss}(Puppi) nominal [GeV]", nbins = 12, xarray = np.array([100, 125, 150, 175, 200, 225, 250, 275, 300, 350, 400, 500, 1000], dtype = 'd')))#,xmin = 100, xmax=800))
var.append(variable(name = "MHT", title= "MHT [GeV]", nbins = 6, xarray = np.array([0, 100, 200, 300, 400, 500, 1000], dtype = 'd')))#,xmin = 100, xmax=800))
var.append(variable(name = "PuppiMET_T1_phi_nominal", title= "Puppi MET #phi nominal", nbins = 6, xmin = -math.pi, xmax=math.pi))
var.append(variable(name = "nJet", title= "# Jet", nbins = 10, xmin = -0.5, xmax=9.5))
var.append(variable(name = "nFatJet", title= "# FatJet", nbins = 5, xmin = -0.5, xmax=4.5))
var.append(variable(name = "LeadingElectronPt_pt", title= "Leading Electron p_{T} [GeV]", nbins = 30, xmin = 0, xmax=300))
var2d = [] 
var2d.append(variable2D(name = "PuppiMET_T1_pt_nominalVsMHT", xname = "PuppiMET_T1_pt_nominal", yname = "MHT", xtitle = " Puppi MET p_{T} [GeV]", ytitle = "MHT [GeV]", nxbins = 12, xarray = np.array([100, 125, 150, 175, 200, 225, 250, 275, 300, 350, 400, 500, 1000], dtype = 'd'),
                            nybins = 6, yarray = np.array([0, 100, 200, 300, 400, 500, 1000], dtype = 'd')))

print("Variables for histograms :")
print([v._name for v in var])

datasets = []
for in_d in in_dataset:
    if not in_d in sample_dict.keys():
        print("Check the in_dataset string... ", sample_dict.keys())
    else : 
        datasets.append(sample_dict[in_d])
print("Datasets to process : ", [d.label for d in datasets])


chain = {}
ntot_events = {}
for d in datasets:
    if hasattr(d, "components"):
        samples_list = d.components
    else:
        samples_list = [d]
    chain[d.label] = {}
    ntot_events[d.label] = {}
    for s in samples_list:
        if distributed: 
            nfiles = len(samples[d.label][s.label]['strings'])
            for i, string in enumerate(samples[d.label][s.label]['strings']): 
                samples[d.label][s.label]['strings'][i] = string.replace("root://cms-xrd-global.cern.ch/", "root://stormgf2.pi.infn.it/")
            chain[d.label][s.label] = samples[d.label][s.label]['strings']
        else: 
            nfiles = nfiles_max
            chain[d.label][s.label] = samples[d.label][s.label]['strings'][:nfiles]
        if not "Data" in s.label: ntot_events[d.label][s.label] = np.sum(samples[d.label][s.label]['ntot'][:nfiles])
        else: ntot_events[d.label][s.label] = None
        print("Dataset : "+s.label)
        print("# of files to process : ", nfiles)
        if distributed and len(chain[d.label][s.label])>2:
            print("files strings :\n  {}\n  {}\n  ... \n  {}\n  {}".format(chain[d.label][s.label][0], chain[d.label][s.label][1], chain[d.label][s.label][-2], chain[d.label][s.label][-1]))
        else :
            print("files strings :\n  {}".format(chain[d.label][s.label][0]))
        print("# of total events in the files to process (MC only, if Data the number is None) : ", ntot_events[d.label][s.label])

Regions to book: 
  orthogonalPreselR_Ntot
  orthogonalPreselR_Npass
  orthogonalPreselR2_Ntot
  orthogonalPreselR2_Npass
  orthogonalPreselR_CR
Variables for histograms :
['PuppiMET_pt', 'PuppiMET_phi', 'PuppiMET_T1_pt_nominal', 'MHT', 'PuppiMET_T1_phi_nominal', 'nJet', 'nFatJet', 'LeadingElectronPt_pt']
Datasets to process :  ['QCD_2022']
Dataset : QCD_HT70to100_2022
# of files to process :  122
files strings :
  root://stormgf2.pi.infn.it//store/user/acagnott/Run3Analysis_Tprime/QCD_HT70to100_2022/20240704_122343/tree_hadd_23.root
  root://stormgf2.pi.infn.it//store/user/acagnott/Run3Analysis_Tprime/QCD_HT70to100_2022/20240704_122343/tree_hadd_95.root
  ... 
  root://stormgf2.pi.infn.it//store/user/acagnott/Run3Analysis_Tprime/QCD_HT70to100_2022/20240704_122343/tree_hadd_5.root
  root://stormgf2.pi.infn.it//store/user/acagnott/Run3Analysis_Tprime/QCD_HT70to100_2022/20240704_122343/tree_hadd_86.root
# of total events in the files to process (MC only, if Data the number is None) :  18

In [5]:
# invLepveto = ""#"!" # per non invertirlo basta passare stinga vuota
# met_cut = 250
# mdphi_cut = 0
# HLT_filter = "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60 || HLT_PFMETNoMu120_PFMHTNoMu120_IDTight || HLT_Ele32_WPTight_Gsf || HLT_Ele115_CaloIdVT_GsfTrkIdT || HLT_Photon200 || HLT_IsoMu24"

In [6]:
# initialization of clusters

# upload the proxyfile to the Dask workers to make them able to access data on the grid 

from distributed.diagnostics.plugin import UploadFile
def set_proxy(dask_worker):
    import os
    import shutil
    working_dir = dask_worker.local_directory
    proxy_name = 'x509up_u0'
    os.environ['X509_USER_PROXY'] = working_dir + '/' + proxy_name
    os.environ['X509_CERT_DIR']="/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
    shutil.copyfile(working_dir + '/' + proxy_name, working_dir + '/../../../proxy')    
    os.environ['EXTRA_CLING_ARGS'] = "-O2"
    return os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR")

text_file = open("utils/postselection.h", "r")
data = text_file.read()
def my_initialization_function():
    print(ROOT.gInterpreter.ProcessLine(".O"))
    ROOT.gInterpreter.Declare('{}'.format(data))
    print("end of initialization")

# set up everything properly
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    client.restart()
    client.register_worker_plugin(UploadFile("/tmp/x509up_u0"))
    client.run(set_proxy)
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)
else:
    RDataFrame = ROOT.RDataFrame
    my_initialization_function()

0
end of initialization


In [7]:
################### utils ###################
def cut_string(cut):
    return cut.replace(" ", "").replace("&&","_").replace(">","_g_").replace(".","_").replace("==","_e_")

################### preselection ###############
def preselection(df, btagAlg, year, EE):
    
    # df = df.Filter("MET_pt>250")
    
    df = df.Define("GoodJet_idx", "GetGoodJet(Jet_pt_nominal, Jet_eta, Jet_jetId)")
    df = df.Define("nGoodJet", "nGoodJet(GoodJet_idx)")
    df = df.Define("GoodFatJet_idx", "GetGoodJet(FatJet_pt_nominal, FatJet_eta, FatJet_jetId)")
    df = df.Define("nGoodFatJet", "GoodFatJet_idx.size()")
    df = df.Filter("nGoodJet>2 || nGoodFatJet>0 ", "jet presel")
    
    # df = df.Redefine("MaxEta_jet", "max_etajet(Jet_eta, GoodJet_idx)")
    df = df.Redefine("MinDelta_phi", "min_DeltaPhi(PuppiMET_T1_phi_nominal, Jet_phi, GoodJet_idx)")
    
    df = df.Define("nTightElectron", "nTightElectron(Electron_pt, Electron_eta, Electron_cutBased)")
    df = df.Define("TightElectron_idx", "TightElectron_idx(Electron_pt, Electron_eta, Electron_cutBased)")
    df = df.Define("nVetoElectron", "nVetoElectron(Electron_pt, Electron_cutBased, Electron_eta)")
    df = df.Define("nTightMuon", "nTightMuon(Muon_pt, Muon_eta, Muon_tightId)")
    df = df.Define("TightMuon_idx", "TightMuon_idx(Muon_pt, Muon_eta, Muon_tightId)")
    df = df.Define("nVetoMuon", "nVetoMuon(Muon_pt, Muon_eta, Muon_looseId)")
    df = df.Define("Lepton_flavour", "Lepton_flavour(nTightElectron, nTightMuon)").Define("Lep_pt", "Lepton_var(Lepton_flavour, Electron_pt, TightElectron_idx, Muon_pt, TightMuon_idx)").Define("Lep_phi", "Lepton_var(Lepton_flavour, Electron_phi, TightElectron_idx, Muon_phi, TightMuon_idx)")
    df = df.Define("MT", "sqrt(2 * Lep_pt * PuppiMET_T1_pt_nominal * (1 - cos(Lep_phi - PuppiMET_T1_phi_nominal)))")
    df = df.Define("MHT", "MHT(GoodJet_idx, Jet_pt, Jet_phi, Jet_eta, Jet_mass)")

    df = df.Define("LeadingJetPt_idx", "GetLeadingPtJet(Jet_pt_nominal)")
    df = df.Define("LeadingJetPt_pt", "GetLeadingJetVar(LeadingJetPt_idx, Jet_pt_nominal)")
    df = df.Define("LeadingJetPt_eta", "GetLeadingJetVar(LeadingJetPt_idx, Jet_eta)")
    df = df.Define("LeadingJetPt_phi", "GetLeadingJetVar(LeadingJetPt_idx, Jet_phi)")
    df = df.Define("LeadingJetPt_mass", "GetLeadingJetVar(LeadingJetPt_idx, Jet_mass_nominal)")
    df = df.Define("LeadingFatJetPt_idx", "GetLeadingPtJet(FatJet_pt)")
    df = df.Define("LeadingFatJetPt_pt", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_pt_nominal)")
    df = df.Define("LeadingFatJetPt_eta", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_eta)")
    df = df.Define("LeadingFatJetPt_phi", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_phi)")
    df = df.Define("LeadingFatJetPt_mass", "GetLeadingJetVar(LeadingFatJetPt_idx, FatJet_mass_nominal)")
    df = df.Define("LeadingMuonPt_idx", "GetLeadingPtLep(Muon_pt, Muon_eta, Muon_looseId)")
    df = df.Define("LeadingMuonPt_pt", "GetLeadingJetVar(LeadingMuonPt_idx, Muon_pt)")
    df = df.Define("LeadingMuonPt_eta", "GetLeadingJetVar(LeadingMuonPt_idx, Muon_eta)")
    df = df.Define("LeadingMuonPt_phi", "GetLeadingJetVar(LeadingMuonPt_idx, Muon_phi)")
    df = df.Define("LeadingElectronPt_idx", "GetLeadingPtLep(Electron_pt, Electron_eta, Electron_cutBased)")
    df = df.Define("LeadingElectronPt_pt", "GetLeadingJetVar(LeadingElectronPt_idx, Electron_pt)")
    df = df.Define("LeadingElectronPt_eta", "GetLeadingJetVar(LeadingElectronPt_idx, Electron_eta)")
    df = df.Define("LeadingElectronPt_phi", "GetLeadingJetVar(LeadingElectronPt_idx, Electron_phi)")
    
    df = df.Define("nForwardJet", "nForwardJet(Jet_pt_nominal, Jet_jetId, Jet_eta)")
    # df = df.Define("JetBTag_idx", "GetJetBTag(GoodJet_idx, "+bTagAlg+","+str(year)+","+str(EE)+")")\
    #             .Define("nJetBtag", "static_cast<int>(JetBTag_idx.size());")
    
    return df

############### trigger selection #####################
def trigger_filter(df, data, isMC):
    hlt_met = "(HLT_PFMET120_PFMHT120_IDTight || HLT_PFMETNoMu120_PFMHTNoMu120_IDTight)"
    # hlt_mu  = "(HLT_IsoMu24 || HLT_Mu50)"
    hlt_ele = "(HLT_Ele32_WPTight_Gsf || HLT_Ele115_CaloIdVT_GsfTrkIdT || HLT_Photon200)"
    df_trig = df.Filter(hlt_met+" || "+hlt_ele, "trigger")
    # if isMC:
    #     df_trig = df.Filter(hlt_met+" || "+hlt_ele, "triggerMET")
    # else:
    #     if "MET" in data:
    #         df_trig = df.Filter(hlt_met, "triggerMET")
    #     # elif "Mu" in data:
    #     #     df_trig = df.Filter("!"+hlt_met+" && "+hlt_mu, "triggerMu")
    #     elif "EG" in data:
    #         df_trig = df.Filter("!"+hlt_met+" && "+hlt_ele, "triggerEle")
    return df_trig

############### top selection ########################
def select_top(df, isMC):
    # return indices of the FatJet with particleNet score over the thresholds 
    df_goodtopMer = df.Define("GoodTopMer_idx", "select_TopMer(FatJet_particleNetWithMass_TvsQCD)")
    # return indices of the TopMixed over the threshold with any object in common
    df_goodtopMix = df_goodtopMer.Define("GoodTopMix_idx", "select_TopMix(TopMixed_TopScore, TopMixed_idxFatJet, TopMixed_idxJet0, TopMixed_idxJet1, TopMixed_idxJet2)")
    # return indices of the TopResolved over the threshold with any object in common
    df_goodtopRes = df_goodtopMix.Define("GoodTopRes_idx", "select_TopRes(TopResolved_TopScore, TopResolved_idxJet0, TopResolved_idxJet1, TopResolved_idxJet2)")
    
    df_nTops = df_goodtopRes.Define("nGoodTopResolved", "nTop(GoodTopRes_idx)")\
                            .Define("nGoodTopMixed", "nTop(GoodTopMix_idx)")\
                            .Define("nGoodTopMerged", "nTop(GoodTopMer_idx)")
    
    
    # return:  1- Event Resolved, 2- Event Mixed, 3- Event Merged, 4- Event Nothing, ...
    df_topcategory = df_nTops.Define("EventTopCategory", "select_TopCategory(GoodTopMer_idx, GoodTopMix_idx, GoodTopRes_idx)")
    if isMC:
        df_topcategory = df_topcategory.Define("EventTopCategoryWithTruth", "select_TopCategoryWithTruth(EventTopCategory, FatJet_matched, GoodTopMer_idx, TopMixed_truth, GoodTopMix_idx, TopResolved_truth, GoodTopRes_idx)")
    
    df_topselected = df_topcategory.Define("Top_idx",
                                           "select_bestTop(EventTopCategory, FatJet_particleNetWithMass_TvsQCD, TopMixed_TopScore, TopResolved_TopScore)")
    # return best top idx wrt category --> the idx is referred to the list of candidates fixed by the EventTopCategory
    df_topvariables = df_topselected.Define("Top_pt", "select_TopVar(EventTopCategory, Top_idx, FatJet_pt_nominal, TopMixed_pt, TopResolved_pt)")\
                        .Define("Top_eta", "select_TopVar(EventTopCategory, Top_idx, FatJet_eta, TopMixed_eta, TopResolved_eta)")\
                        .Define("Top_phi", "select_TopVar(EventTopCategory, Top_idx, FatJet_phi, TopMixed_phi, TopResolved_phi)")\
                        .Define("Top_mass", "select_TopVar(EventTopCategory, Top_idx, FatJet_mass_nominal, TopMixed_mass, TopResolved_mass)")\
                        .Define("Top_score", "select_TopVar(EventTopCategory, Top_idx, FatJet_particleNetWithMass_TvsQCD, TopMixed_TopScore, TopResolved_TopScore)")\
                        .Define("Top_isolationPtJetsdR04","TopIsolation_NJets(EventTopCategory, Top_idx, TopMixed_idxFatJet, TopMixed_idxJet0, TopMixed_idxJet1, TopMixed_idxJet2, TopMixed_pt, TopMixed_phi, TopMixed_eta, TopResolved_idxJet0, TopResolved_idxJet1, TopResolved_idxJet2, TopResolved_pt, TopResolved_phi, TopResolved_eta, FatJet_pt_nominal, FatJet_eta, FatJet_phi, FatJet_jetId, Jet_pt_nominal, Jet_eta, Jet_phi, Jet_jetId, 0.4, 1)")\
                        .Define("Top_isolationPtJetsdR06","TopIsolation_NJets(EventTopCategory, Top_idx, TopMixed_idxFatJet, TopMixed_idxJet0, TopMixed_idxJet1, TopMixed_idxJet2, TopMixed_pt, TopMixed_phi, TopMixed_eta, TopResolved_idxJet0, TopResolved_idxJet1, TopResolved_idxJet2, TopResolved_pt, TopResolved_phi, TopResolved_eta, FatJet_pt_nominal, FatJet_eta, FatJet_phi, FatJet_jetId, Jet_pt_nominal, Jet_eta, Jet_phi, Jet_jetId, 0.6, 1)")\
                        .Define("Top_isolationPtJetsdR08","TopIsolation_NJets(EventTopCategory, Top_idx, TopMixed_idxFatJet, TopMixed_idxJet0, TopMixed_idxJet1, TopMixed_idxJet2, TopMixed_pt, TopMixed_phi, TopMixed_eta, TopResolved_idxJet0, TopResolved_idxJet1, TopResolved_idxJet2, TopResolved_pt, TopResolved_phi, TopResolved_eta, FatJet_pt_nominal, FatJet_eta, FatJet_phi, FatJet_jetId, Jet_pt_nominal, Jet_eta, Jet_phi, Jet_jetId, 0.8, 1)")\
                        .Define("Top_isolationPtJetsdR12","TopIsolation_NJets(EventTopCategory, Top_idx, TopMixed_idxFatJet, TopMixed_idxJet0, TopMixed_idxJet1, TopMixed_idxJet2, TopMixed_pt, TopMixed_phi, TopMixed_eta, TopResolved_idxJet0, TopResolved_idxJet1, TopResolved_idxJet2, TopResolved_pt, TopResolved_phi, TopResolved_eta, FatJet_pt_nominal, FatJet_eta, FatJet_phi, FatJet_jetId, Jet_pt_nominal, Jet_eta, Jet_phi, Jet_jetId, 1.2, 1)")\
                        .Define("Top_isolationNJetsdR04","TopIsolation_NJets(EventTopCategory, Top_idx, TopMixed_idxFatJet, TopMixed_idxJet0, TopMixed_idxJet1, TopMixed_idxJet2, TopMixed_pt, TopMixed_phi, TopMixed_eta, TopResolved_idxJet0, TopResolved_idxJet1, TopResolved_idxJet2, TopResolved_pt, TopResolved_phi, TopResolved_eta, FatJet_pt_nominal, FatJet_eta, FatJet_phi, FatJet_jetId, Jet_pt_nominal, Jet_eta, Jet_phi, Jet_jetId, 0.4, 0)")\
                        .Define("Top_isolationNJetsdR06","TopIsolation_NJets(EventTopCategory, Top_idx, TopMixed_idxFatJet, TopMixed_idxJet0, TopMixed_idxJet1, TopMixed_idxJet2, TopMixed_pt, TopMixed_phi, TopMixed_eta, TopResolved_idxJet0, TopResolved_idxJet1, TopResolved_idxJet2, TopResolved_pt, TopResolved_phi, TopResolved_eta, FatJet_pt_nominal, FatJet_eta, FatJet_phi, FatJet_jetId, Jet_pt_nominal, Jet_eta, Jet_phi, Jet_jetId, 0.6, 0)")\
                        .Define("Top_isolationNJetsdR08","TopIsolation_NJets(EventTopCategory, Top_idx, TopMixed_idxFatJet, TopMixed_idxJet0, TopMixed_idxJet1, TopMixed_idxJet2, TopMixed_pt, TopMixed_phi, TopMixed_eta, TopResolved_idxJet0, TopResolved_idxJet1, TopResolved_idxJet2, TopResolved_pt, TopResolved_phi, TopResolved_eta, FatJet_pt_nominal, FatJet_eta, FatJet_phi, FatJet_jetId, Jet_pt_nominal, Jet_eta, Jet_phi, Jet_jetId, 0.8, 0)")\
                        .Define("Top_isolationNJetsdR12","TopIsolation_NJets(EventTopCategory, Top_idx, TopMixed_idxFatJet, TopMixed_idxJet0, TopMixed_idxJet1, TopMixed_idxJet2, TopMixed_pt, TopMixed_phi, TopMixed_eta, TopResolved_idxJet0, TopResolved_idxJet1, TopResolved_idxJet2, TopResolved_pt, TopResolved_phi, TopResolved_eta, FatJet_pt_nominal, FatJet_eta, FatJet_phi, FatJet_jetId, Jet_pt_nominal, Jet_eta, Jet_phi, Jet_jetId, 1.2, 0)")

    if isMC:
        df_topvariables = df_topvariables.Define("Top_truth", "select_TopVar(EventTopCategory, Top_idx, FatJet_matched, TopMixed_truth, TopResolved_truth)")
    # NB: TopTruth for Merged is replaced with FatJet_matched, the variable is between 0 and 3 
    # where 3 means true end less than 3 means false 
    return df_topvariables

In [8]:
def bookhisto(df, regions_def, var, s_cut):
    h_ = {}
    for reg in regions_def.keys():
        h_[reg] = {}
        for v in var:
            if "SFbtag" in v._name: continue

            if v._MConly and not sampleflag: 
                continue
            else:
                if regions_def[reg] == "":
                    if v._xarray is None:
                        h_[reg][v._name]= df.Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name, "w_nominal")
                    else:
                        h_[reg][v._name]= df.Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xarray), v._name, "w_nominal")
                else:
                    if v._xarray is None:
                        # print(v._name, reg, regions_def[reg])
                        h_[reg][v._name]= df.Filter(regions_def[reg]).Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name, "w_nominal")
                    else:
                        h_[reg][v._name]= df.Filter(regions_def[reg]).Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xarray), v._name, "w_nominal")                    
    
    return h_

def bookhisto2D(df, regions_def, var2d, s_cut):
    h_ = {}
    for reg in regions_def.keys():
        h_[reg] = {}
        for v in var2d:
            if regions_def[reg]=="":
                if v._xarray is None:
                    h_[reg][v._name] = df.Redefine(v._xname, "UnOvBin("+v._xname+","+str(v._nxbins)+","+str(v._xmin)+","+str(v._xmax)+")")\
                                     .Redefine(v._yname, "UnOvBin("+v._yname+","+str(v._nybins)+","+str(v._ymin)+","+str(v._ymax)+")")\
                                     .Histo2D((v._xname+"Vs"+v._yname+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xmin, v._xmax, v._nybins, v._ymin, v._ymax), v._xname, v._yname, "w_nominal")
                else:
                    h_[reg][v._name] = df.Redefine(v._xname, "UnOvBin("+v._xname+","+str(v._nxbins)+","+str(v._xarray[0])+","+str(v._xarray[-1])+")")\
                                     .Redefine(v._yname, "UnOvBin("+v._yname+","+str(v._nybins)+","+str(v._ymin)+","+str(v._ymax)+")")\
                                     .Histo2D((v._xname+"Vs"+v._yname+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xarray, v._nybins, v._ymin, v._ymax), v._xname, v._yname, "w_nominal")
            else:
                if v._xarray is None:
                    h_[reg][v._name] = df.Filter(regions_def[reg])\
                                     .Redefine(v._xname, "UnOvBin("+v._xname+","+str(v._nxbins)+","+str(v._xmin)+","+str(v._xmax)+")")\
                                     .Redefine(v._yname, "UnOvBin("+v._yname+","+str(v._nybins)+","+str(v._ymin)+","+str(v._ymax)+")")\
                                     .Histo2D((v._xname+"Vs"+v._yname+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xmin, v._xmax, v._nybins, v._ymin, v._ymax), v._xname, v._yname, "w_nominal")
                else:
                    h_[reg][v._name] = df.Filter(regions_def[reg])\
                                     .Redefine(v._xname, "UnOvBin("+v._xname+","+str(v._nxbins)+","+str(v._xarray[0])+","+str(v._xarray[-1])+")")\
                                     .Redefine(v._yname, "UnOvBin("+v._yname+","+str(v._nybins)+","+str(v._yarray[0])+","+str(v._yarray[-1])+")")\
                                     .Histo2D((v._xname+"Vs"+v._yname+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xarray, v._nybins, v._yarray), v._xname, v._yname, "w_nominal")
    return h_

In [9]:
def savehisto(d, h, regions_def, var, s_cut, isMC):
    histo = {reg: {v._name: ROOT.TH1D(v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax) if v._xarray is None else ROOT.TH1D(v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xarray) for v in var} for reg in regions_def.keys()}
    isMC=True
    if "Data" in d.label: isMC = False
    if hasattr(d, "components"):
        s_list = d.components
    else:
        s_list = [d]
    
    for s in s_list:
        outfile = ROOT.TFile.Open(repohisto+s.label+'.root', "RECREATE")
        for reg in regions_def.keys():
            for v in var:
                if "SFbtag" in v._name: continue
                if v._MConly and not isMC:
                    continue
                else:
                    histo[reg][v._name] = h[d.label][s.label][reg][v._name].GetValue()
                    if isMC:
                        histo[reg][v._name].Scale(s.sigma*10**3/ntot_events[d.label][s.label])
                    outfile.cd()
                    histo[reg][v._name].Write()
        outfile.Close()

# i plot2d per il momento non ci servono, si deve trovare un modo più intelligente di farli
def savehisto2d(d, h, regions_def, var2d, s_cut, isMC):
    histo = {reg: {v._name: ROOT.TH2D(v._name+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xmin, v._xmax, v._nybins, v._ymin, v._ymax)  if v._xarray is None else ROOT.TH2D(v._name+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xarray, v._nybins, v._yarray)  for v in var2d} for reg in regions_def.keys()}
        
    if hasattr(d, "components"):
        s_list = d.components
    else:
        s_list = [d]
    
    for s in s_list:
        outfile = ROOT.TFile.Open(repohisto+s.label+'_2D.root', "RECREATE")
        for reg in regions_def.keys():
            for v in histo[reg].keys():
                histo[reg][v] = h[d.label][s.label][reg][v].GetValue()
                if isMC:
                    histo[reg][v].Scale(s.sigma*10**3/ntot_events[d.label][s.label])
                outfile.cd()
                histo[reg][v].Write()
        outfile.Close()

In [10]:
#samples['DataHTF_2022']['DataHTF_2022']['strings']

In [11]:
#samples[d.label][d.components[0]]['strings']

In [12]:
t0 = datetime.now()
print("starting loop on datasets: ",[d.label for d in datasets])
print("Local time :", t0)
# print("requirements: "+cut)

h = {}
h_2D = {}

for d in datasets:
    if hasattr(d, "components"):
        s_list = d.components
    else:
        s_list = [d]

    if 'Data' in d.label : sampleflag = 0
    else: sampleflag = 1
    c_ = cut
    h[d.label] = {}
    h_2D[d.label] = {}
    for s in s_list:
        #-------------------------------------------------------------------------
        ############# Fixing variables for 2018-2022 #############################
        #-------------------------------------------------------------------------
        if s.year == 2018:
            bTagAlg = "Jet_btagDeepB"
        elif s.year == 2022:
            bTagAlg = "Jet_btagPNetB"
        if hasattr(s,"EE"):
            EE = s.EE
        else:
            EE = 0
        #-------------------------------------------------------------------------
        #########################  DF initialization #############################
        #-------------------------------------------------------------------------
        
        print("Initializing DataFrame for "+ s.label +" chain len = ", len(chain[d.label][s.label]))
        if len(chain[d.label][s.label])==1: print(chain[d.label][s.label])
        if distributed ==True:
            df = RDataFrame("Events", chain[d.label][s.label], npartitions=nmaxpartition, 
                            daskclient=client, monitor_label = "main" )
        else:
            df = RDataFrame("Events", chain[d.label][s.label])
        
        
        df_ismc         = df.Define("isMC", "isMC("+str(sampleflag)+")")
        df_year         = df_ismc.Define("year", str(s.year))
        df_hemveto      = df_year.Define("HEMVeto", "hemveto(Jet_eta, Jet_phi, Electron_eta, Electron_phi)")
        df_hemveto      = df_hemveto.Filter("(isMC || (year != 2018) || (HEMVeto || run<319077.))")
        df_hlt          = trigger_filter(df_hemveto, s.label, sampleflag)
        ### per gli HLT potremmo volerli modificare a seconda della regione (?) in tal caso la funzione
        ### va spostata nel book histos e va passata la regione per attivare trigger diversi
        if sampleflag: df_wnom = df_hlt.Define('w_nominal', 'puWeight*SFbtag_nominal') # controllare questi pesi nominal 
        else: df_wnom           = df_hlt.Define('w_nominal', '1')
        # df_wnom           = df_hlt.Define('w_nominal', '1')
        df_presel       = preselection(df_wnom, bTagAlg, s.year, EE)
        # df_topsel       = select_top(df_presel, sampleflag)
        
        
        if do_snapshot:
            opts = ROOT.RDF.RSnapshotOptions()
            opts.fLazy = True
            if distributed: fold = "./"
            else: fold = folder
            snapshot_df = df_presel.Snapshot("events_nominal", fold+"snap_"+s.label+".root", branches, opts)
            # print("./"+s.label+".root")
        if do_histos:
            s_cut = cut_string(cut)
            if len(var) != 0 :
                h[d.label][s.label] = bookhisto(df_presel, regions_def, var, s_cut)
            if len(var2d) != 0 :
                h_2D[d.label][s.label] = bookhisto2D(df_presel, regions_def, var2d, s_cut)
# if not distributed:
#     df_presel.Report().Print()

if do_histos:
    print("All histos booked !")
    for d in datasets:
        if len(var):
            savehisto(d, h, regions_def, var, s_cut, sampleflag)
        if len(var2d) != 0 :
            savehisto2d(d, h_2D, regions_def, var2d, s_cut, sampleflag)
        print(d.label + " histos saved")
if do_snapshot:
    snapshot_df.GetValue()
    if distributed: 
        client.run(transfer_to_tier)
        print("Snapshots saved and trasfered to tier")
    print("Sanpshot done!")
t1 = datetime.now()
print("Job finished in: ", t1-t0)

starting loop on datasets:  ['QCD_2022']
Local time : 2024-07-16 10:33:43.447329
Initializing DataFrame for QCD_HT70to100_2022 chain len =  122
Initializing DataFrame for QCD_HT100to200_2022 chain len =  84
Initializing DataFrame for QCD_HT200to400_2022 chain len =  113
Initializing DataFrame for QCD_HT400to600_2022 chain len =  99
Initializing DataFrame for QCD_HT600to800_2022 chain len =  118
Initializing DataFrame for QCD_HT800to1000_2022 chain len =  105
Initializing DataFrame for QCD_HT1000to1200_2022 chain len =  66
Initializing DataFrame for QCD_HT1200to1500_2022 chain len =  135
Initializing DataFrame for QCD_HT1500to2000_2022 chain len =  88
Initializing DataFrame for QCD_HT2000_2022 chain len =  110
All histos booked !
QCD_2022 histos saved
Job finished in:  0:18:29.643750


Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available
Error in <TList::Clear>: A list is accessing an object (0xb16f890) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xb1c8060) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xb14a670) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xb1cd460) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x9447d30) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessin

In [13]:
file = ROOT.TFile.Open(repohisto+s.label+".root")
print(file)
# for a in file.GetListOfKeys(): print(a)
for reg in regions_def.keys():
    # for v in var:
    hist = file.Get(var[0]._name+"_"+reg+"_")
    print(reg, hist.Integral())

Name: ./results/run2022_trigSF/plots/QCD_HT2000_2022.root Title: 
orthogonalPreselR_Ntot 1.5178682358924744
orthogonalPreselR_Npass 0.6005578831585214
orthogonalPreselR2_Ntot 1.5178682358924744
orthogonalPreselR2_Npass 0.0
orthogonalPreselR_CR 0.027021846651267008


Error in <TList::Clear>: A list is accessing an object (0xd37d9b0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xad3a750) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xda15cf0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xda27490) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xd349cf0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0xd3728a0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x8167030) already deleted (list name = TList)


In [14]:
# regions_def.keys()

In [15]:
# region = "NoCut"
# var = "MET_pt"

In [16]:
if do_snapshot and distributed:
    print(os.popen("davix-ls davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{}/{}/ -E /tmp/x509up_u0 --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/".format(remote_folder_name, remote_subfolder_name)).read())

In [17]:
# c1 = ROOT.TCanvas()
# c1.Draw()
hist = file.Get("PuppiMET_T1_pt_nominal_"+reg+"_")
hist.GetNbinsX()

12

In [18]:
for i in range(1, 11): print(hist.GetXaxis().GetBinLowEdge(i),hist.GetXaxis().GetBinUpEdge(i))

100.0 125.0
125.0 150.0
150.0 175.0
175.0 200.0
200.0 225.0
225.0 250.0
250.0 275.0
275.0 300.0
300.0 350.0
350.0 400.0


In [19]:
# h0 = file.Get("PuppiMET_T1_pt_nominal_orthogonalPreselR_Ntot_")
# c1 = ROOT.TCanvas()
# c1.Draw()
# h0.Draw()